# Views are different perspectives of our data
That would be a way to get a better understanding of the data.  
If you have an idea for a view, share it or make an issue.

In [ ]:
import altair as alt
import numpy as np
import pandas as pd

from mlrepricer import match, setup, helper, schemas
from mlrepricer.minmax import dump_csv
from mlrepricer.database import SQLite
alt.data_transformers.enable('default', max_rows=1000000)

In [ ]:
t = schemas.pricemonitor(SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')

### Filter1 aka features we probably should not track in the first place.
for me it drops like 4% of the data

In [ ]:
# this is available in mlrepricer.helper.cleanup()
filter1 = df[(df.instock==1)&(df.isfeaturedmerchant==1)]
filter1 = filter1.drop(['instock', 'isfeaturedmerchant'], axis=1)
# those offers suck too
filter1 = filter1[(filter1.shipping_maxhours+filter1.shipping_minhours)<=72]

We look at each attribute separetely now.  
Basically it's ok to drop both features.  
The instock attribute has some winners, but it's to unlikely.  
It's like 1/300 of the dataset got that feature.

In [ ]:
drop2 = df[df.isfeaturedmerchant==0]  # it has a zero chance to be a winner
print(f'We can drop {len(drop2)} rows and loose {len(drop2[drop2.isbuyboxwinner==1])} winners, we expected this to be Zero')

In [ ]:

drop = df[df.instock==0]
normalized_lostwinner = len(filter1[filter1.isbuyboxwinner==1])/len(filter1)/(len(drop[drop.isbuyboxwinner==1])/len(drop))
print(f'Its {normalized_lostwinner:.2f} times less likely to find a buyboxwinner in the data set we drop here, we have {len(drop)} datarows.')
if normalized_lostwinner < 5:
    raise ValueError

# The stuff below is not wrong, but it makes only little sense to look at the data this way.
Better just look on a per message level. Because one message has all the information for a state.

## What is the name for the type of feed data we get?
It's like time series data, but tracking only changes.
I think without reducing the complexity we can imagine it in 5 dimensions.
 - asin
 - time_changed  
   competitor = groupby(['sellerid', 'isprime']).min()  # min because maybe some have duplicates offers  
     those are two seperate dimensions:
     - sellerid
     - isprime
 - features
 
does this make sense?

Knowing this we can make a pivot table. Let's refer to it as groundtruth.

In [ ]:
filter1.groupby(['asin', 'sellerid', 'isprime', 'time_changed']).min().head(10)

What can we improve from here?
 - only track stable states, like those hold for more than 5 minutes
 - linear transform isprime to remove a dimension  
More ideas are welcome on github.